<a href="https://colab.research.google.com/github/anirbanmukherjee2709/tsai_end2.0_Session_1/blob/main/Session_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

There are no possible combinations to have 44 parameters with 2 input neurons and 1 output neuron and only 1 single hidden layer

In [ ]:
display(' 1 hidden layer')
display([f'in: {i}, hidden1: {j}, out: {k}' for i in range(1, 10) for j in range(1, 10) for k in range(1, 10) if j*(1+i) + k*(1+j) == 44 and k < i and k < j])

' 1 hidden layer'

['in: 3, hidden1: 7, out: 2', 'in: 4, hidden1: 6, out: 2']

Trying with 2 hidden layers

In [ ]:
display(' 2 hidden layer')
display([f'in: {2}, hidden_1: {j}, hidden_2: {k}, Out: {1}' for j in range(1, 10) for k in range(1, 10) if j*(1+2) + k*(1+j) + 1*(1+k) == 44])


' 2 hidden layer'

['in: 2, hidden_1: 5, hidden_2: 4, Out: 1']

There is only one possible combinations to have 44 parameters with 2 input neurons and 1 output neuron and only 2 hidden layers. Hence, we implement the same

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
torch.manual_seed(2)

In [ ]:
X = torch.Tensor([[0,0], [0,1], [1,0], [1,1]])
Y = torch.Tensor([0, 1, 1, 0]).view(-1,1)

In [ ]:
class XOR(nn.Module):
    def __init__(self, input_dim = 2, output_dim=1):
        super(XOR, self).__init__()
        self.lin1 = nn.Linear(input_dim,5)
        self.lin2 = nn.Linear(5, 4)
        self.lin3 = nn.Linear(4, output_dim)
    
    def forward(self, x):
        x = self.lin1(x)
        x = F.tanh(x)
        x = self.lin2(x)
        x = F.tanh(x)
        x = self.lin3(x)
        # x = F.tanh(x) # removing the last activation
        return x

In [ ]:
model = XOR()
print(model)
from torchsummary import summary
summary(model, (2, 2))

XOR(
  (lin1): Linear(in_features=2, out_features=5, bias=True)
  (lin2): Linear(in_features=5, out_features=4, bias=True)
  (lin3): Linear(in_features=4, out_features=1, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 2, 5]              15
            Linear-2                 [-1, 2, 4]              24
            Linear-3                 [-1, 2, 1]               5
Total params: 44
Trainable params: 44
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [ ]:
def weights_init(model):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            # initialize the weight tensor, here we use a normal distribution
            m.weight.data.normal_(0, 1)

weights_init(model)

In [ ]:
loss_func = nn.L1Loss()

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.9)

In [ ]:
epochs = 2001
steps = X.size(0)
for i in range(epochs):
    for j in range(steps):
        data_point = np.random.randint(X.size(0))
        x_var = Variable(X[data_point], requires_grad=False)
        y_var = Variable(Y[data_point], requires_grad=False)
        
        optimizer.zero_grad()
        y_hat = model(x_var)
        loss = loss_func.forward(y_hat, y_var)
        loss.backward()
        optimizer.step()
        
    if i % 50 == 0:
        print( "Epoch: {0}, Loss: {1}, ".format(i + 1, loss.data.numpy()))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch: 1, Loss: 1.852978229522705, 
Epoch: 51, Loss: 0.24401819705963135, 
Epoch: 101, Loss: 0.09616148471832275, 
Epoch: 151, Loss: 0.27723217010498047, 
Epoch: 201, Loss: 0.01733708381652832, 
Epoch: 251, Loss: 0.281794011592865, 
Epoch: 301, Loss: 0.12283611297607422, 
Epoch: 351, Loss: 0.1630803346633911, 
Epoch: 401, Loss: 0.5191493630409241, 
Epoch: 451, Loss: 0.12391293048858643, 
Epoch: 501, Loss: 0.030515074729919434, 
Epoch: 551, Loss: 0.0027251243591308594, 
Epoch: 601, Loss: 0.24932122230529785, 
Epoch: 651, Loss: 0.11529660224914551, 
Epoch: 701, Loss: 0.3468332290649414, 
Epoch: 751, Loss: 0.025042057037353516, 
Epoch: 801, Loss: 0.039009809494018555, 
Epoch: 851, Loss: 0.0723467469215393, 
Epoch: 901, Loss: 0.03940308094024658, 
Epoch: 951, Loss: 0.1568109393119812, 
Epoch: 1001, Loss: 0.05481874942779541, 
Epoch: 1051, Loss: 0.34722596406936646, 
Epoch: 1101, Loss: 0.24947881698608398, 
Epoch: 1151, Loss: 0.2845611572265625, 
Epoch: 1201, Loss: 0.059189677238464355, 
Ep